In [1]:
import pandas as pd
import json
from collections import defaultdict
from tqdm import tqdm
import random
from copy import deepcopy
#!pip install zhconv
import zhconv,re

# 读取KG

In [185]:
with open('/data/aisearch/nlp/data/xhsun/KG/kgClue/Knowledge.txt') as f:
    lines=f.readlines()

print('The number of triples: {}'.format(len(lines)))

sub_map = defaultdict(set)#每一个头实体作为key，对应的所有一跳路径内的(关系，尾实体)作为value
# so_map=defaultdict(set)#每一对(头实体，尾实体)作为key，对应的所有一跳的关系作为value
sp_map=defaultdict(set)#每一个（头实体，关系）作为key，对应的仅能有一个答案

alias_map=defaultdict(set)
ent_to_relations=defaultdict(set)
bad_line=0
for i in tqdm(range(len(lines))):
    line=lines[i]
    l = line.strip().split('\t')
    s = l[0].strip()
    p = l[1].strip()
    o = l[2].strip()
    if s=='' or p=='' or o=='':
        bad_line+=1
        continue
    sub_map[s].add((p, o))
#     so_map[(s,o)].add(p)
    sp_map[(s,p)].add(o)

    ent_to_relations[s].add(p)

    entity_mention=s
    if '（' in s and '）' in s:
        entity_mention=s.split('（')[0]
        alias_map[entity_mention].add(s)
    if p in ['别名','中文名','英文名','昵称','中文名称','英文名称','别称','全称','原名']:
        alias_map[entity_mention].add(o)

  0%|          | 85/22884223 [00:00<53:57:19, 117.81it/s]

The number of triples: 22884223


100%|██████████| 22884223/22884223 [02:26<00:00, 156700.15it/s]


In [3]:
bad_line

674

# 读取测试数据

In [38]:
with open('/data/aisearch/nlp/data/xhsun/KG/QA_data/kgclue/official_data/test.json') as f:
    lines=f.readlines()
test_examples=[]
for line in lines:
    test_examples.append(json.loads(line.strip()))

# 加载模型

In [40]:
import sys,os,torch
sys.path.append(os.path.abspath('../'))
from bart import MyBart
bart_model_path='/data/aisearch/nlp/data/xhsun/huggingfaceModels/chinese/chinese-bart-base'
checkpoint='../../saved_models/kgqa/mixture/best-model.pt'
model=MyBart.from_pretrained(bart_model_path,state_dict=torch.load(checkpoint,map_location='cpu'))

In [41]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(bart_model_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'BertTokenizer'.


# 首先测试NER

In [19]:
for i in range(len(test_examples)):
    example=test_examples[i]
    demo_q=example['question']
    demo_q='{}'.format(demo_q)+'。上个句子中哪些属于实体名词？_。'
    test_examples[i].update({'input_q_ner':demo_q})

In [31]:
for i in tqdm(range(len(test_examples)),total=len(test_examples),unit='sentence'):
    example=test_examples[i]
    demo_q=example['input_q_ner']
    batch=tokenizer.batch_encode_plus([demo_q])
    outputs = model.generate(input_ids=torch.LongTensor(batch['input_ids']),
                             attention_mask=torch.LongTensor(batch['attention_mask']),
                             num_beams=4,
                             min_length=1,
                             max_length=30,
                             early_stopping=True,)
    ner_results=tokenizer.decode(outputs[0],skip_special_tokens=True,clean_up_tokenization_spaces=True)#.strip()
    test_examples[i].update({"topic_entity":ner_results})
    print(test_examples[i])

  0%|          | 2/3000 [00:00<07:04,  7.06sentence/s]

{'id': 0, 'question': '我很好奇刘质平的老师是谁？', 'input_q_ner': '我很好奇刘质平的老师是谁？。上个句子中哪些属于实体名词？_。', 'topic_entity': '刘 质 平'}
{'id': 1, 'question': '请问南关町的平假名是什么？', 'input_q_ner': '请问南关町的平假名是什么？。上个句子中哪些属于实体名词？_。', 'topic_entity': '南 关 町'}


  0%|          | 4/3000 [00:00<06:36,  7.55sentence/s]

{'id': 2, 'question': '请问新道寺女子高中的平假名是什么？', 'input_q_ner': '请问新道寺女子高中的平假名是什么？。上个句子中哪些属于实体名词？_。', 'topic_entity': '新 道 寺 女 子 高 中'}
{'id': 3, 'question': '海部市的平假名是什么？', 'input_q_ner': '海部市的平假名是什么？。上个句子中哪些属于实体名词？_。', 'topic_entity': '海 部 市'}
{'id': 4, 'question': '八代市的平假名是什么？', 'input_q_ner': '八代市的平假名是什么？。上个句子中哪些属于实体名词？_。', 'topic_entity': '八 代 市'}


  0%|          | 7/3000 [00:00<06:12,  8.04sentence/s]

...

{'id': 2994, 'question': '告诉我中国少数民族文学的学科代码', 'input_q_ner': '告诉我中国少数民族文学的学科代码。上个句子中哪些属于实体名词？_。', 'topic_entity': '中 国 少 数 民 族 文 学'}
{'id': 2995, 'question': '那么中国民间文学的学科代码是什么呢', 'input_q_ner': '那么中国民间文学的学科代码是什么呢。上个句子中哪些属于实体名词？_。', 'topic_entity': '中 国 民 间 文 学'}


100%|█████████▉| 2999/3000 [07:03<00:00,  9.28sentence/s]

{'id': 2996, 'question': '那么中国现代文学的学科代码是什么呢', 'input_q_ner': '那么中国现代文学的学科代码是什么呢。上个句子中哪些属于实体名词？_。', 'topic_entity': '中 国 现 代 文 学'}
{'id': 2997, 'question': '1900的挚友是谁', 'input_q_ner': '1900的挚友是谁。上个句子中哪些属于实体名词？_。', 'topic_entity': '1900'}
{'id': 2998, 'question': '丁丁的挚友是谁', 'input_q_ner': '丁丁的挚友是谁。上个句子中哪些属于实体名词？_。', 'topic_entity': '丁 丁'}


100%|██████████| 3000/3000 [07:03<00:00,  7.08sentence/s]

{'id': 2999, 'question': '哪个是三岛仁八的挚友', 'input_q_ner': '哪个是三岛仁八的挚友。上个句子中哪些属于实体名词？_。', 'topic_entity': '三 岛 仁 八'}


In [44]:
sub_map['C2001次']

{('中文名', 'C2001次'),
 ('全程时间', '30分钟'),
 ('列车等级', '城际高速'),
 ('列车配属', '京局京段'),
 ('开车时间', '06:33'),
 ('总里程', '120KM'),
 ('所属铁路局', '北京铁路局'),
 ('终到时间', '07:03'),
 ('运行区段', '北京南-天津'),
 ('途经省市', '北京市、天津市'),
 ('途经线路', '京津高速铁路')}

In [18]:
import sys,os,torch
sys.path.append(os.path.abspath('../'))
from src.utils import read_test_data
from src.rule_tools import rule1,only_en_char,check_has_en

In [47]:
def rule1_for_find_ner(topic_entity,question,sub_map,alias_map):
    '''
    以topic_entity为起始点，往右边找
    '''
    assert topic_entity in question
    start_idx=question.find(topic_entity)
    end_idx=start_idx+len(topic_entity)
    for i in range(end_idx,len(question)):
        if question[start_idx:i] in sub_map or question[start_idx:i] in alias_map:
            topic_entity=question[start_idx:i]
            break
    
    return topic_entity

def rule2_for_find_ner(topic_entity,question,sub_map,alias_map):
    '''
    以topic_entity为终止点，往左边找
    '''
    assert topic_entity in question
    start_idx=question.find(topic_entity)
    end_idx=start_idx+len(topic_entity)
    for i in range(0,start_idx+1):
        if question[i:end_idx] in sub_map or question[i:end_idx] in alias_map:
            topic_entity=question[i:end_idx]
            break
    
    return topic_entity

def rule3_for_find_ner(topic_entity,question,sub_map,alias_map):
    '''
    以topic_entity中心，从左到右删除字符
    '''
    assert topic_entity in question
    for i in range(len(topic_entity)):
        if (topic_entity[i:] in sub_map or topic_entity[i:] in alias_map):
            topic_entity=topic_entity[i:]
            break
    return topic_entity

def rule4_for_find_ner(topic_entity,question,sub_map,alias_map):
    '''
    以topic_entity中心，从右到左删除字符
    '''
    assert topic_entity in question
    for i in range(len(topic_entity),1):
        if topic_entity[:i] in sub_map or topic_entity[:i] in alias_map:
            topic_entity=topic_entity[:i]
            break
    return topic_entity


In [48]:
del alias_map['']

In [73]:
ner_results=read_test_data('../outputs/ner_results.json')

The number of test examples : 3000


In [74]:
ner_results

[{'id': 0,
  'question': '我很好奇刘质平的老师是谁？',
  'input_q_ner': '我很好奇刘质平的老师是谁？。上个句子中哪些属于实体名词？_。',
  'pred_ner': '刘 质 平'},
 {'id': 1,
  'question': '请问南关町的平假名是什么？',
  'input_q_ner': '请问南关町的平假名是什么？。上个句子中哪些属于实体名词？_。',
  'pred_ner': '南 关 町'},
 {'id': 2,
  'question': '请问新道寺女子高中的平假名是什么？',
  'input_q_ner': '请问新道寺女子高中的平假名是什么？。上个句子中哪些属于实体名词？_。',
  'pred_ner': '新 道 寺 女 子 高 中'},
 {'id': 3,
  'question': '海部市的平假名是什么？',
  'input_q_ner': '海部市的平假名是什么？。上个句子中哪些属于实体名词？_。',
  'pred_ner': '海 部 市'},
 {'id': 4,
  'question': '八代市的平假名是什么？',
  'input_q_ner': '八代市的平假名是什么？。上个句子中哪些属于实体名词？_。',
  'pred_ner': '八 代 市'},
 {'id': 5,
  'question': '三泽市的平假名叫什么？',
  'input_q_ner': '三泽市的平假名叫什么？。上个句子中哪些属于实体名词？_。',
  'pred_ner': '三 泽 市'},
 {'id': 6,
  'question': '芥川龙之介奖的平假名叫什么？',
  'input_q_ner': '芥川龙之介奖的平假名叫什么？。上个句子中哪些属于实体名词？_。',
  'pred_ner': '芥 川 龙 之 介 奖'},
 {'id': 7,
  'question': '尤莉的日本字怎么写啊？',
  'input_q_ner': '尤莉的日本字怎么写啊？。上个句子中哪些属于实体名词？_。',
  'pred_ner': '尤 莉'},
 {'id': 8,
  'question': '临海女子高中的日本字怎么写啊？',
  'input_q_ner

In [76]:
def get_topic_entity(ner_results,sub_map,alias_map,write_results_path):
    '''
    有如下问题需要考虑：
    （1）实体名词全部是英文：
            question中是大写字母，但是由于tokenizer是uncased，所以输出的答案是小写字母
            由于是英文，所以不能进行''.join(xx.split())操作
    （2）实体名词全部是中文：
            可以直接进行''.join(xx.split())操作，需要注意繁体字的问题
    （3）实体名词中既有中文又有英文
    '''
    if '' in sub_map:
        del sub_map['']
    if '' in alias_map:
        del alias_map['']
    assert '' not in sub_map and '' not in alias_map

    wrong_ner_results=[]
    for i in range(len(ner_results)):
        example=ner_results[i]
        question=example['question']
        pred_ner=example['pred_ner']
        topic_entity=pred_ner
        if check_has_en(pred_ner):
            #有英文字母，可能全是英文字母也可能中英文混合
    #         if only_en_char(topic_entity):
    #             #全是英文字母
            if topic_entity not in question:
                #有英文字母但是这个实体不在问题中，那么可能存在大小写问题，应该以question中的字母为准
                temp_question=question.lower()
                temp_entity=topic_entity.lower()
                if temp_entity in temp_question:
                    start_idx=temp_question.find(temp_entity)
                    end_idx=start_idx+len(temp_entity)
                    topic_entity=question[start_idx:end_idx]

        #默认现在是仅有中文的情况
        if topic_entity not in question:
            topic_entity=''.join(topic_entity.split())
            
            if check_has_en(topic_entity) and topic_entity not in question:
                #此时说明既有中文又有英文，但是这个实体不在问题中，那么可能存在大小写问题，应该以question中的字母为准
                temp_question=question.lower()
                temp_entity=topic_entity.lower()
                if temp_entity in temp_question:
                    start_idx=temp_question.find(temp_entity)
                    end_idx=start_idx+len(temp_entity)
                    topic_entity=question[start_idx:end_idx]
            
            if topic_entity not in question and zhconv.convert(topic_entity,'zh-hans') in question:
                topic_entity=zhconv.convert(topic_entity,'zh-hans')
                
        if topic_entity not in question:
            #还是没有找到
            topic_entity=rule1(topic_entity=pred_ner,question=question)

        if topic_entity not in question:
            wrong_ner_results.append(example)

        ner_results[i].update({"topic_entity":topic_entity})

    not_found_in_question=0
    for example in ner_results:
        if example['topic_entity'] not in example['question']:
            not_found_in_question+=1
    assert not_found_in_question==len(wrong_ner_results)

    print("wrong_ner_results : {}".format(len(wrong_ner_results)))
    for example in wrong_ner_results:
        print(example)
        print('-'*100)

    ##################现在的阶段是，有4个识别的主题实体在question中根本找不到，2996个实体在question中可以找得到
    ##################但是有一些识别的实体不在KG中
    for i in range(len(ner_results)):
        example=ner_results[i]
        topic_entity=example['topic_entity']
        question=example['question']
        if topic_entity not in question:
            print("{} not in {}".format(topic_entity,question))
            continue
        if  topic_entity not in sub_map and topic_entity not in alias_map:
            topic_entity=rule1_for_find_ner(topic_entity=topic_entity,question=question,sub_map=sub_map,alias_map=alias_map)
        
        if  topic_entity not in sub_map and topic_entity not in alias_map:
            topic_entity=rule2_for_find_ner(topic_entity=topic_entity,question=question,sub_map=sub_map,alias_map=alias_map)
        
        if  topic_entity not in sub_map and topic_entity not in alias_map:
            topic_entity=rule3_for_find_ner(topic_entity=topic_entity,question=question,sub_map=sub_map,alias_map=alias_map)
        
        if  topic_entity not in sub_map and topic_entity not in alias_map:
            topic_entity=rule4_for_find_ner(topic_entity=topic_entity,question=question,sub_map=sub_map,alias_map=alias_map)
        
        ner_results[i].update({"topic_entity":topic_entity})
    
    not_found_in_kg=0
    for example in ner_results:
        if example['topic_entity'] not in sub_map and example['topic_entity'] not in alias_map:
            not_found_in_kg+=1
            print("example : not found entity in KG", example)

    print('not_found_in_kg : {}'.format(not_found_in_kg))
    
    with open(write_results_path,'w') as f:
        for example in ner_results:
            idx=example['id']
            topic_entity=example['topic_entity']
            question=example['question']
            f.write(json.dumps({"id":idx,'question':question,'topic_entity':topic_entity},ensure_ascii=False)+'\n')
    
    print('Entity has been written in {}'.format(write_results_path))


In [77]:
get_topic_entity(ner_results,sub_map,alias_map,write_results_path='../outputs/ner_results.json')

wrong_ner_results : 4
{'id': 640, 'question': '耿庄村有哪些主导产业？', 'input_q_ner': '耿庄村有哪些主导产业？。上个句子中哪些属于实体名词？_。', 'pred_ner': '郸 庄 村', 'topic_entity': '郸 庄 村'}
----------------------------------------------------------------------------------------------------
{'id': 1851, 'question': '你知道007之最高机密是是哪个等级吗', 'input_q_ner': '你知道007之最高机密是是哪个等级吗。上个句子中哪些属于实体名词？_。', 'pred_ner': '007 系 列', 'topic_entity': '007 系 列'}
----------------------------------------------------------------------------------------------------
{'id': 2283, 'question': "哪部剧接档了Healin' Good ♡ 光之美少女", 'input_q_ner': "哪部剧接档了Healin' Good ♡ 光之美少女。上个句子中哪些属于实体名词？_。", 'pred_ner': "healin'good ♡ 光 之 美 少 女", 'topic_entity': "healin'good ♡ 光 之 美 少 女"}
----------------------------------------------------------------------------------------------------
{'id': 2457, 'question': 'Boom Boom Boom第一次演出是什么时候', 'input_q_ner': 'Boom Boom Boom第一次演出是什么时候。上个句子中哪些属于实体名词？_。', 'pred_ner': 'boom boom poom', 'topic_entity': 'boom boom poom'}
----------------

In [78]:
ner_results=read_test_data('../outputs/ner_results.json')

The number of test examples : 3000


In [79]:
ner_results

[{'id': 0, 'question': '我很好奇刘质平的老师是谁？', 'topic_entity': '刘质平'},
 {'id': 1, 'question': '请问南关町的平假名是什么？', 'topic_entity': '南关町'},
 {'id': 2, 'question': '请问新道寺女子高中的平假名是什么？', 'topic_entity': '新道寺女子高中'},
 {'id': 3, 'question': '海部市的平假名是什么？', 'topic_entity': '海部市'},
 {'id': 4, 'question': '八代市的平假名是什么？', 'topic_entity': '八代市'},
 {'id': 5, 'question': '三泽市的平假名叫什么？', 'topic_entity': '三泽市'},
 {'id': 6, 'question': '芥川龙之介奖的平假名叫什么？', 'topic_entity': '芥川龙之介奖'},
 {'id': 7, 'question': '尤莉的日本字怎么写啊？', 'topic_entity': '尤莉'},
 {'id': 8, 'question': '临海女子高中的日本字怎么写啊？', 'topic_entity': '临海女子高中'},
 {'id': 9, 'question': '我很好奇子实体是什么构造的？', 'topic_entity': '子实体'},
 {'id': 10, 'question': '我很好奇涨紧轮是什么构造的？', 'topic_entity': '涨紧轮'},
 {'id': 11, 'question': '你知道卡板的构造吗？', 'topic_entity': '卡板'},
 {'id': 12, 'question': '你知道苑里车站的构造吗？', 'topic_entity': '苑里车站'},
 {'id': 13, 'question': '你知道急流槽的主要构成吗？', 'topic_entity': '急流槽'},
 {'id': 14, 'question': '你知道煌斑岩的主要构成吗？', 'topic_entity': '煌斑岩'},
 {'id': 15, 'question': '涅墨西斯

In [84]:
inputs_for_kg=[]

for i in range(len(ner_results)):
    example=ner_results[i]
    idx=example['id']
    topic_entity=example['topic_entity']
    question=example['question']
    candidate_questions=[]
    for ent in alias_map[topic_entity]:
        relations=ent_to_relations[ent]
        if relations==set():
            continue
        candidate_relations=list(relations)
        candidate_relations.append('不匹配')
        random.shuffle(candidate_relations)#确保“不匹配”这个选项不总是在最后一个位置

        relation_string=[]
        for i,rel in enumerate(candidate_relations):
            relation_string.append('（{}）'.format(i)+rel)
        relation_string='，'.join(relation_string)
        kg_q='{}'.format(question)+'。'+'实体名词是：{}'.format(ent)+"。这个句子的意图与下列哪一个关系最相似？_。"+' '+relation_string
        candidate_questions.append(kg_q)
        
    #test_examples[i].update({"candidate_questions":deepcopy(candidate_questions)})
    inputs_for_kg.append({"id":idx,'question':question,'topic_entity':topic_entity,'candidate_questions':candidate_questions})

In [85]:
len(inputs_for_kg)

3000

In [115]:
for input_q in demo_example['candidate_questions']:
    print(re.findall("实体名词是：(.*)。?",input_q))

['寨后社区。这个句子的意图与下列哪一个关系最相似？_。 （0）中文名称，（1）不匹配，（2）农户，（3）乡村人口，（4）行政区类别，（5）所属地区']
['寨后社区（云南省丽江市古城区西安街道下辖社区）。这个句子的意图与下列哪一个关系最相似？_。 （0）行政区类别，（1）中文名，（2）乡村人口，（3）所属地区，（4）不匹配，（5）农户']
['寨后社区（福建省厦门市海沧区东孚街道下辖社区）。这个句子的意图与下列哪一个关系最相似？_。 （0）中文名，（1）所属地区，（2）地理位置，（3）不匹配']


In [94]:
candidate_questions=demo_example['candidate_questions']

In [101]:
batch=tokenizer.batch_encode_plus(candidate_questions,pad_to_max_length=True,max_length=384)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/cdou/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [102]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print('Using device : {}'.format(device))
model=model.to(device)
outputs = model.generate(input_ids=torch.LongTensor(batch['input_ids']).to(device=device),
                        attention_mask=torch.LongTensor(batch['attention_mask']).to(device=device),
                        num_beams=4,
                        min_length=1,
                        max_length=30,
                        early_stopping=True,)

Using device : cuda


In [104]:
tokenizer.decode(outputs[0],skip_special_tokens=True,clean_up_tokenization_spaces=True)

'乡 村 人 口'

In [108]:
tokenizer.decode(outputs[1],skip_special_tokens=True,clean_up_tokenization_spaces=True)

'人 口'

In [109]:
tokenizer.decode(outputs[2],skip_special_tokens=True,clean_up_tokenization_spaces=True)

'不 匹 配'

In [106]:
sub_map['寨后社区（云南省丽江市古城区西安街道下辖社区）']

{('中文名', '寨后社区'),
 ('乡村人口', '859人'),
 ('农户', '149户'),
 ('所属地区', '云南省丽江市古城区西安街道办事处'),
 ('行政区类别', '社区')}

In [260]:
qa_results=read_test_data('../outputs/qa_results.json')

The number of test examples : 3000


In [261]:
qa_results[:10]

[{'id': 0,
  'question': '我很好奇刘质平的老师是谁？',
  'topic_entity': '刘质平',
  'candidate_questions': ['我很好奇刘质平的老师是谁？。实体名词是：刘质平。这个句子的意图与下列哪一个关系最相似？_。 （0）中文名，（1）出生日期，（2）师从，（3）不匹配，（4）国籍，（5）毕业院校，（6）主要成就，（7）师兄弟，（8）出生地，（9）代表作品，（10）职业，（11）逝世日期'],
  'candidate_combinations': [['我很好奇刘质平的老师是谁？。实体名词是：刘质平。这个句子的意图与下列哪一个关系最相似？_。 （0）中文名，（1）出生日期，（2）师从，（3）不匹配，（4）国籍，（5）毕业院校，（6）主要成就，（7）师兄弟，（8）出生地，（9）代表作品，（10）职业，（11）逝世日期',
    '师 从']]},
 {'id': 1,
  'question': '请问南关町的平假名是什么？',
  'topic_entity': '南关町',
  'candidate_questions': ['请问南关町的平假名是什么？。实体名词是：南关町。这个句子的意图与下列哪一个关系最相似？_。 （0）所属地区，（1）不匹配，（2）平假名，（3）外文名，（4）中文名'],
  'candidate_combinations': [['请问南关町的平假名是什么？。实体名词是：南关町。这个句子的意图与下列哪一个关系最相似？_。 （0）所属地区，（1）不匹配，（2）平假名，（3）外文名，（4）中文名',
    '平 假 名']]},
 {'id': 2,
  'question': '请问新道寺女子高中的平假名是什么？',
  'topic_entity': '新道寺女子高中',
  'candidate_questions': ['请问新道寺女子高中的平假名是什么？。实体名词是：新道寺女子高中。这个句子的意图与下列哪一个关系最相似？_。 （0）外文名，（1）地区，（2）中文名，（3）不匹配，（4）平假名，（5）简称，（6）罗马音，（7）层次'],
  'candidate_combinations': [['请问新道寺女子高中的平假名是什么？。实体名词是：新道寺女子高中。这个句

In [262]:
def char_match_answer(question,ent_in_kg,sub_map):
    for r,o in sub_map[ent_in_kg]:
        if r in question:
            return o
    return ''

char_match_examples=[]
#     #每一个别名实体，找到这个别名实体的所有关系，如果某一个关系可以在question找到对应的字符，那么也就不需要其它操作了

#     found=False
#     for ent_in_kg in alias_entities:
#         o=char_match_answer(question,ent_in_kg=ent_in_kg,sub_map=sub_map)
#         if o!='':
#             found=True
#             break
#     if found:
#         char_match_examples.append((question,topic_entity,ent_in_kg,o))


submit_examples=[]
for i in range(len(qa_results)):
    example=qa_results[i]
    question=example['question']
    candidate_combinations=example['candidate_combinations']
    topic_entity=example['topic_entity']
    alias_entities=list(alias_map[topic_entity])
    if topic_entity not in alias_entities:
        alias_entities.append(topic_entity)
        
    predict_answers=[]
    for each_possible_answer in candidate_combinations:
        input_q_with_prompt,predict_r=each_possible_answer
        ent_in_kg=re.findall('实体名词是：(.*)。这个句子的意图与下列哪一个关系最相似？_。',input_q_with_prompt)[0]
        predict_r=''.join(predict_r.split())
        
        if predict_r.lower() in input_q_with_prompt.lower():
            start_idx=input_q_with_prompt.lower().find(predict_r.lower())
            end_idx=start_idx+len(predict_r)
            predict_r=input_q_with_prompt[start_idx:end_idx]
            
        if predict_r=='不匹配':
            continue
        if predict_r in input_q_with_prompt:
            #此时input_q_with_prompt中的实体名词和predict_r就构成了答案
            for p,o in sub_map[ent_in_kg]:
                if p==predict_r:
                    break
            predict_answers.append((ent_in_kg,predict_r,o))
#         if check_has_en(predict_r):
#             #关系中有英文字母存在
    #print(question,predict_answers)
    submit_examples.append({'id':example['id'],'question':question,'topic_entity':topic_entity,'predict_answers':predict_answers})

In [263]:
len(submit_examples)

3000

In [264]:
submit_examples

[{'id': 0,
  'question': '我很好奇刘质平的老师是谁？',
  'topic_entity': '刘质平',
  'predict_answers': [('刘质平', '师从', '李叔同')]},
 {'id': 1,
  'question': '请问南关町的平假名是什么？',
  'topic_entity': '南关町',
  'predict_answers': [('南关町', '平假名', 'なんかんまち')]},
 {'id': 2,
  'question': '请问新道寺女子高中的平假名是什么？',
  'topic_entity': '新道寺女子高中',
  'predict_answers': [('新道寺女子高中', '平假名', 'しんどうじじょし')]},
 {'id': 3,
  'question': '海部市的平假名是什么？',
  'topic_entity': '海部市',
  'predict_answers': [('海部市', '平假名', 'あまし')]},
 {'id': 4,
  'question': '八代市的平假名是什么？',
  'topic_entity': '八代市',
  'predict_answers': [('八代市', '平假名', 'やつしろし')]},
 {'id': 5,
  'question': '三泽市的平假名叫什么？',
  'topic_entity': '三泽市',
  'predict_answers': [('三泽市', '平假名', 'みさわし')]},
 {'id': 6,
  'question': '芥川龙之介奖的平假名叫什么？',
  'topic_entity': '芥川龙之介奖',
  'predict_answers': [('芥川龙之介奖', '平假名', 'あくたがわりゅうのすけしょう')]},
 {'id': 7,
  'question': '尤莉的日本字怎么写啊？',
  'topic_entity': '尤莉',
  'predict_answers': []},
 {'id': 8,
  'question': '临海女子高中的日本字怎么写啊？',
  'topic_entity': '临海女子高中',
  'pre

In [265]:
alias_map['卡萨布兰卡']

{'卡萨布兰卡',
 '卡萨布兰卡（1973年美国唱片公司）',
 '卡萨布兰卡（关燕仪音乐专辑）',
 '卡萨布兰卡（千妍演唱歌曲）',
 '卡萨布兰卡（百合科百合属植物）',
 '卡萨布兰卡（百合花品种）',
 '卡萨布兰卡（美国1942年迈克尔·柯蒂斯执导电影）',
 '卡萨布兰卡（群星演唱歌曲）',
 '卡萨布兰卡（贝蒂·希金斯演唱歌曲）',
 '卡萨布兰卡（音乐舞台剧）',
 '卡萨布兰卡（黄莺莺演唱歌曲）',
 '香水百合',
 '香水百合、百合之王'}

In [266]:
sub_map['卡萨布兰卡（百合科百合属植物）']

{('中文名', '卡萨布兰卡'),
 ('分布区域', '澳洲'),
 ('别名', '香水百合、百合之王'),
 ('属', '百合属'),
 ('拉丁学名', 'Lilium casa blanca'),
 ('界', '植物界'),
 ('目', '百合目'),
 ('科', '百合科'),
 ('纲', '单子叶植物纲'),
 ('花期', '夏'),
 ('英文别称', 'Lily'),
 ('门', '被子植物门')}

In [267]:
submissions=[]
multiple_answers=[]
not_found_answers=[]
for example in submit_examples:
    predict_answers=example['predict_answers']
    topic_entity=example['topic_entity']
    
    if predict_answers!=[]:
        if len(predict_answers)>1:
            multiple_answers.append(example)
        predict_answer=predict_answers[0]
        s,p,o=predict_answer
        answer=' ||| '.join([s,p,o])
    else:
        not_found_answers.append(example)
        answer=' ||| '.join([topic_entity,topic_entity,topic_entity])
    submissions.append({"id":example['id'],'question':example['question'],'answer':answer})

In [268]:
submissions

[{'id': 0, 'question': '我很好奇刘质平的老师是谁？', 'answer': '刘质平 ||| 师从 ||| 李叔同'},
 {'id': 1, 'question': '请问南关町的平假名是什么？', 'answer': '南关町 ||| 平假名 ||| なんかんまち'},
 {'id': 2,
  'question': '请问新道寺女子高中的平假名是什么？',
  'answer': '新道寺女子高中 ||| 平假名 ||| しんどうじじょし'},
 {'id': 3, 'question': '海部市的平假名是什么？', 'answer': '海部市 ||| 平假名 ||| あまし'},
 {'id': 4, 'question': '八代市的平假名是什么？', 'answer': '八代市 ||| 平假名 ||| やつしろし'},
 {'id': 5, 'question': '三泽市的平假名叫什么？', 'answer': '三泽市 ||| 平假名 ||| みさわし'},
 {'id': 6,
  'question': '芥川龙之介奖的平假名叫什么？',
  'answer': '芥川龙之介奖 ||| 平假名 ||| あくたがわりゅうのすけしょう'},
 {'id': 7, 'question': '尤莉的日本字怎么写啊？', 'answer': '尤莉 ||| 尤莉 ||| 尤莉'},
 {'id': 8,
  'question': '临海女子高中的日本字怎么写啊？',
  'answer': '临海女子高中 ||| 临海女子高中 ||| 临海女子高中'},
 {'id': 9, 'question': '我很好奇子实体是什么构造的？', 'answer': '子实体 ||| 构造 ||| 高等真菌'},
 {'id': 10,
  'question': '我很好奇涨紧轮是什么构造的？',
  'answer': '涨紧轮 ||| 构造 ||| 主要由固定壳体、张紧臂组成'},
 {'id': 11, 'question': '你知道卡板的构造吗？', 'answer': '卡板 ||| 构造 ||| 川字型'},
 {'id': 12,
  'question': '你知道苑里车站的构造吗？',
  'answer': '

In [269]:
sub_map['海戛村（云南曲靖市富源县营上镇下辖村）']

{('中文名称', '海戛村'), ('乡村人口', '3072人'), ('海拔', '1800米'), ('行政区类别', '村')}

In [270]:
with open('../outputs/submissions.json','w') as f:
    for example in submissions:
        f.write(json.dumps(example,ensure_ascii=False)+'\n')

In [256]:
sub_map['急流槽']

{('中文名', '急流槽'),
 ('作用', '在距离短、水面落差大时引导水流'),
 ('外文名', 'chute'),
 ('学科', '公路交通科学技术'),
 ('定义', '坡度大于临界坡度的人工沟槽'),
 ('构造', '进口、槽身和出口')}

In [271]:
len(not_found_answers)

147

In [272]:
len(multiple_answers)

207

In [221]:
sub_map['陈石']

set()

In [222]:
{"id": 405, 
 "question": "请问陈石的出生死亡年份是多久？", 
 "topic_entity": "陈石", 
 "candidate_questions": ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（泸县税务局党委委员、纪检组长、二级主办）。这个句子的意图与下列哪一个关系最相似？_。 （0）性别，（1）民族，（2）出生日期，（3）籍贯，（4）不匹配，（5）政治面貌，（6）中文名", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（水利工程建设局政治处处长）。这个句子的意图与下列哪一个关系最相似？_。 （0）不匹配，（1）民族，（2）性别，（3）出生日期，（4）中文名", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（贵州省六盘水市林业局局长）。这个句子的意图与下列哪一个关系最相似？_。 （0）不匹配，（1）出生日期，（2）中文名，（3）国籍，（4）性别", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（原广东机械学校校长）。这个句子的意图与下列哪一个关系最相似？_。 （0）生卒，（1）中文名，（2）曾用名，（3）籍贯，（4）不匹配", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（革命烈士）。这个句子的意图与下列哪一个关系最相似？_。 （0）逝世日期，（1）主要成就，（2）不匹配，（3）出生日期，（4）国籍，（5）中文名，（6）民族，（7）出生地，（8）性别，（9）职业", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（卓越新能独立董事）。这个句子的意图与下列哪一个关系最相似？_。 （0）职务，（1）性别，（2）学历，（3）不匹配，（4）公司", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（政协贵阳市第十一届委员会原主席）。这个句子的意图与下列哪一个关系最相似？_。 （0）出生地，（1）中文名，（2）学历，（3）性别，（4）出生日期，（5）不匹配", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（水城区委书记、水城经济开发区党工委书记）。这个句子的意图与下列哪一个关系最相似？_。 （0）政治面貌，（1）出生日期，（2）毕业院校，（3）中文名，（4）民族，（5）籍贯，（6）职务，（7）国籍，（8）不匹配", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（宝应县副县长）。这个句子的意图与下列哪一个关系最相似？_。 （0）不匹配，（1）任职单位，（2）职务，（3）中文名", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（山东画院画师）。这个句子的意图与下列哪一个关系最相似？_。 （0）民族，（1）中文名，（2）不匹配，（3）国籍，（4）性别", "请问陈石的出生死亡年份是多久？。实体名词是：陈石（倍腾学校身亡学员）。这个句子的意图与下列哪一个关系最相似？_。 （0）中文名，（1）国籍，（2）不匹配，（3）职业，（4）民族"], 
 "candidate_combinations": [["请问陈石的出生死亡年份是多久？。实体名词是：陈石（泸县税务局党委委员、纪检组长、二级主办）。这个句子的意图与下列哪一个关系最相似？_。 （0）性别，（1）民族，（2）出生日期，（3）籍贯，（4）不匹配，（5）政治面貌，（6）中文名", "不 匹 配"],
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（水利工程建设局政治处处长）。这个句子的意图与下列哪一个关系最相似？_。 （0）不匹配，（1）民族，（2）性别，（3）出生日期，（4）中文名", "不 匹 配"], 
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（贵州省六盘水市林业局局长）。这个句子的意图与下列哪一个关系最相似？_。 （0）不匹配，（1）出生日期，（2）中文名，（3）国籍，（4）性别", "不 匹 配"],
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（原广东机械学校校长）。这个句子的意图与下列哪一个关系最相似？_。 （0）生卒，（1）中文名，（2）曾用名，（3）籍贯，（4）不匹配", "不 匹 配"], 
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（革命烈士）。这个句子的意图与下列哪一个关系最相似？_。 （0）逝世日期，（1）主要成就，（2）不匹配，（3）出生日期，（4）国籍，（5）中文名，（6）民族，（7）出生地，（8）性别，（9）职业", "不 匹 配"], 
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（卓越新能独立董事）。这个句子的意图与下列哪一个关系最相似？_。 （0）职务，（1）性别，（2）学历，（3）不匹配，（4）公司", "不 匹 配"],
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（政协贵阳市第十一届委员会原主席）。这个句子的意图与下列哪一个关系最相似？_。 （0）出生地，（1）中文名，（2）学历，（3）性别，（4）出生日期，（5）不匹配", "不 匹 配"], 
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（水城区委书记、水城经济开发区党工委书记）。这个句子的意图与下列哪一个关系最相似？_。 （0）政治面貌，（1）出生日期，（2）毕业院校，（3）中文名，（4）民族，（5）籍贯，（6）职务，（7）国籍，（8）不匹配", "不 匹 配"],
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（宝应县副县长）。这个句子的意图与下列哪一个关系最相似？_。 （0）不匹配，（1）任职单位，（2）职务，（3）中文名", "不 匹 配"],
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（山东画院画师）。这个句子的意图与下列哪一个关系最相似？_。 （0）民族，（1）中文名，（2）不匹配，（3）国籍，（4）性别", "不 匹 配"], 
                            ["请问陈石的出生死亡年份是多久？。实体名词是：陈石（倍腾学校身亡学员）。这个句子的意图与下列哪一个关系最相似？_。 （0）中文名，（1）国籍，（2）不匹配，（3）职业，（4）民族", "不 匹 配"]]}

{'陈石',
 '陈石（倍腾学校身亡学员）',
 '陈石（卓越新能独立董事）',
 '陈石（原广东机械学校校长）',
 '陈石（宝应县副县长）',
 '陈石（山东画院画师）',
 '陈石（政协贵阳市第十一届委员会原主席）',
 '陈石（水利工程建设局政治处处长）',
 '陈石（水城区委书记、水城经济开发区党工委书记）',
 '陈石（泸县税务局党委委员、纪检组长、二级主办）',
 '陈石（贵州省六盘水市林业局局长）',
 '陈石（革命烈士）'}

In [218]:
sub_map['学派']

{('中文名', '学派'), ('外文名', 'xue pai'), ('词性', '名词'), ('近义词', '流派')}

212

In [212]:
(3000-212)/3000

0.9293333333333333

In [209]:
len(multiple_answers)

205

In [207]:
alias_map['临海女子高中']

{'临海女子高中'}

In [193]:
alias_map['今日运势']

{'今日运势', '今日运势（今日运势：休闲小游戏）', '今日运势（今日运势：手机软件）', '今日运势（硬糖少女303演唱歌曲）'}

In [196]:
a={'205人'}

In [197]:
a

{'205人'}

In [199]:
a.pop()

'205人'

In [175]:
char_match_examples

[('请问南关町的平假名是什么？', '南关町', '南关町', 'なんかんまち'),
 ('请问新道寺女子高中的平假名是什么？', '新道寺女子高中', '新道寺女子高中', 'しんどうじじょし'),
 ('海部市的平假名是什么？', '海部市', '海部市', 'あまし'),
 ('八代市的平假名是什么？', '八代市', '八代市', 'やつしろし'),
 ('三泽市的平假名叫什么？', '三泽市', '三泽市', 'みさわし'),
 ('芥川龙之介奖的平假名叫什么？', '芥川龙之介奖', '芥川龙之介奖', 'あくたがわりゅうのすけしょう'),
 ('我很好奇子实体是什么构造的？', '子实体', '子实体', '高等真菌'),
 ('我很好奇涨紧轮是什么构造的？', '涨紧轮', '涨紧轮', '主要由固定壳体、张紧臂组成'),
 ('你知道卡板的构造吗？', '卡板', '卡板', '川字型'),
 ('你知道苑里车站的构造吗？', '苑里车站', '苑里车站', '岛式月台、岸式月台各一座'),
 ('涅墨西斯的构造是什么？', '涅墨西斯', '涅墨西斯（动漫《出包王女Darkness》人物）', '暗物质'),
 ('顶棚的构造是什么？', '顶棚', '顶棚（建筑结构名称）', '面层、基层、吊杆'),
 ('大家知道单体壳的构造是什么吗？', '单体壳', '单体壳', '碳纤维'),
 ('大家知道仙台城的构造是什么吗？', '仙台城', '仙台城', '连郭式平山城'),
 ('请问东风6型内燃机车的主要用户有哪些公司？', '东风6型内燃机车', '东风6型内燃机车', '中国铁路总公司'),
 ('请问韶山4型电力机车的主要用户有哪些公司？', '韶山4型电力机车', '韶山4型电力机车', '中国铁路总公司'),
 ('DJ型电力机车的主要用户有谁？', 'DJ型电力机车', 'DJ型电力机车', '中国铁路总公司'),
 ('JR货物EF210型电力机车的主要用户有谁？', 'JR货物EF210型电力机车', 'JR货物EF210型电力机车', '日本货物铁道'),
 ('韶山8型电力机车的主要用户是什么啊？', '韶山8型电力机车', '韶山8型电力机车', '中国铁路总公司'),
 ('九广通的主要用户是什么啊？', '九广通

In [178]:
sub_map['家村']

{('中文名', '家村'),
 ('中文名称', '家村'),
 ('人口', '1548'),
 ('人口数量', '1548'),
 ('外文名', 'JiaVillage'),
 ('外文名称', 'Jia\xa0Village'),
 ('所属地区', '山西省临汾市襄汾县永固乡'),
 ('方言', '晋南方言'),
 ('气候条件', '温带季风气候，四季分明'),
 ('电话区号', '0357'),
 ('耕地面积', '3429余亩'),
 ('著名景点', '村中心屹立两颗百年白杨树，村北边有古庙一座'),
 ('行政区类别', '村'),
 ('车牌代码', '晋L'),
 ('邮政区码', '041506')}

In [180]:
alias_map['朱家村（云南晋宁县二街乡下辖村）']

set()

In [181]:
sub_map['朱家村（云南晋宁县二街乡下辖村）']

{('中文名', '朱家村'), ('国土面积', '23.9平方'), ('年平均气温', '14.6 ℃'), ('海拔', '1919米')}

In [159]:
sub_map[topic_entity]

{('中文名', '三岛仁八'),
 ('主要成就', '三岛财阀的创始人'),
 ('儿子', '三岛平八'),
 ('国籍', '日本'),
 ('性别', '男'),
 ('挚友', '王惊雷')}

In [160]:
#     for each_possible_answer in candidate_combinations:
#         _,predict_r=each_possible_answer
#         if check_has_en(predict_r):
#             print(example)

'三岛仁八'

In [155]:
sub_map['南关町']

{('中文名', '南关町'),
 ('外文名', 'Nankan-machi'),
 ('平假名', 'なんかんまち'),
 ('所属地区', '日本地域九州地方都道府县熊本县郡玉名郡政府')}